In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [2]:
main_url="https://books.toscrape.com"

In [16]:
def book_scrape():
    final_data=[]
    page_url=main_url
    while True:
        total_content=requests.get(page_url)
        data=BeautifulSoup(total_content.text,'html.parser')
        all_books=data.find_all('article',class_='product_pod')
        for i in all_books:
            title=i.h3.a['title']
            price=i.find('p',class_='price_color').text
            stock=i.find('p',class_='instock availability').text.strip()
            rating=i.find('p',class_='star-rating')['class'][1]
            book_href=i.h3.a['href'] #For Entering the Detailed Page to get the genre
            book_url=urljoin(main_url,book_href)
            book_detail=requests.get(book_url)
            book_data=BeautifulSoup(book_detail.text,'html.parser')
            try:
                genre = book_data.select('ul.breadcrumb li a')[2].text.strip()
            except IndexError:
                genre = 'Unknown'
            #genre=book_data.select('ul.breadcrumb li a')[2].text.strip()
            final_data.append({'Title': title,'Price': price,'Rating': rating,'Stock': stock,'Genre': genre})
        next_button = data.find('li', class_='next')
        if next_button:
            next_href = next_button.a['href']
            page_url = urljoin(main_url,next_href)
        else:
            break
    return final_data

In [18]:
last_data=book_scrape()
df=pd.DataFrame(last_data)
df.to_csv('Final_books.csv',index=False)
print("your data is saved to Final_books.csv")
print(df.head())

your data is saved to Final_books.csv
                                   Title    Price Rating     Stock  \
0                   A Light in the Attic  Â£51.77  Three  In stock   
1                     Tipping the Velvet  Â£53.74    One  In stock   
2                             Soumission  Â£50.10    One  In stock   
3                          Sharp Objects  Â£47.82   Four  In stock   
4  Sapiens: A Brief History of Humankind  Â£54.23   Five  In stock   

                Genre  
0              Poetry  
1  Historical Fiction  
2             Fiction  
3             Mystery  
4             History  


In [1]:
#Pyspark

In [3]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

In [3]:
session= SparkSession.builder.appName("sample").getOrCreate()

In [21]:
sa=session.read.csv(r"C:\Users\midhu\Downloads\dm_office_sales (1).csv",header=True, inferSchema=True)

In [22]:
sa.printSchema()

root
 |-- division: string (nullable = true)
 |-- level of education: string (nullable = true)
 |-- training level: integer (nullable = true)
 |-- work experience: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- sales: integer (nullable = true)



In [23]:
sa.show(5)

+---------------+------------------+--------------+---------------+------+------+
|       division|level of education|training level|work experience|salary| sales|
+---------------+------------------+--------------+---------------+------+------+
|       printers|      some college|             2|              6| 91684|372302|
|       printers|associate's degree|             2|             10|119679|495660|
|    peripherals|       high school|             0|              9| 82045|320453|
|office supplies|associate's degree|             2|              5| 92949|377148|
|office supplies|       high school|             1|              5| 71280|312802|
+---------------+------------------+--------------+---------------+------+------+
only showing top 5 rows


In [24]:
sa.describe().show()

+-------+-----------------+------------------+------------------+----------------+------------------+------------------+
|summary|         division|level of education|    training level| work experience|            salary|             sales|
+-------+-----------------+------------------+------------------+----------------+------------------+------------------+
|  count|             1000|              1000|              1000|            1000|              1000|              1000|
|   mean|             NULL|              NULL|             1.262|           6.142|         90264.207|        339833.846|
| stddev|             NULL|              NULL|0.9253173575656154|2.94796989918936|17398.617174354134|108555.77331583285|
|    min|computer hardware|associate's degree|                 0|               0|             41660|             53067|
|    max|         printers|      some college|                 3|              16|            159148|            761477|
+-------+-----------------+-----

In [25]:
sa.filter(sa["salary"] >= 20000).show()

+-----------------+------------------+--------------+---------------+------+------+
|         division|level of education|training level|work experience|salary| sales|
+-----------------+------------------+--------------+---------------+------+------+
|         printers|      some college|             2|              6| 91684|372302|
|         printers|associate's degree|             2|             10|119679|495660|
|      peripherals|       high school|             0|              9| 82045|320453|
|  office supplies|associate's degree|             2|              5| 92949|377148|
|  office supplies|       high school|             1|              5| 71280|312802|
|computer hardware|      some college|             0|             10|102401|366451|
|      peripherals|associate's degree|             0|              5| 77548|230899|
|  office supplies|      some college|             0|              6| 81966|265915|
|         printers|associate's degree|             3|              2| 72844|